# Computing the bicrystallographic aspects of Misorientations (FCC lattice)

In [1]:
# import sys
import gbpy.byxtal.lattice as gbl;
import gbpy.byxtal.csl_utility_functions as cuf;
import gbpy.byxtal.find_csl_dsc as fcd;
import numpy as np;
import numpy.linalg as nla
import gbpy.byxtal.tools as bxt;
import gbpy.byxtal.misorient_fz as mfz;
import gbpy.byxtal.disorient_symm_props as dsp;
# from sympy.matrices import Matrix, eye, zeros;

Define the lattice: Here we use the `lattice` module and use the default option. The default option will initialize a cubic lattice with $a = 1$.

In [2]:
l1 = gbl.Lattice()
sig_type = 'common'
l_p_po = l1.l_p_po

First, initialize the $\Sigma$ number of interest. Note that for cubic crystals, CSL rotations exist only for odd-values of $\Sigma$.

In [3]:
sig_num = 651

The function `csl_rotations` in the module `gbpy.byxtal.csl_utility_functions`, will compute the $\Sigma$ rotation (a misorientation) in the reference frame of the primitive cell.

In [4]:
s1 = cuf.csl_rotations(sig_num, sig_type, l1);
print(s1)

{'N': array([[[ 645.,  -54.,   54.],
        [  74.,  666.,  -15.],
        [ -70.,   21.,  630.]],

       [[ 515., -265.,  149.],
        [ 365.,  716.,   74.],
        [-264.,   60.,  519.]],

       [[ 449., -346.,   44.],
        [ 375.,  681.,   15.],
        [-124.,  155.,  620.]],

       [[ 549., -225.,  141.],
        [ 315.,  714.,   56.],
        [-240.,   45.,  536.]],

       [[ 570., -195.,  174.],
        [ 250.,  634.,  -95.],
        [-161.,  175.,  595.]],

       [[ 495., -286.,  156.],
        [ 420.,  770.,  231.],
        [-396.,  -75.,  396.]],

       [[ 474., -300.,  191.],
        [ 426.,  711.,   92.],
        [-315.,   84.,  476.]],

       [[ 405., -390.,   54.],
        [ 474.,  725.,  150.],
        [-249.,   71.,  531.]],

       [[ 386., -408.,   51.],
        [ 434.,  651.,    0.],
        [-115.,  216.,  624.]],

       [[ 465., -310.,  186.],
        [ 390.,  629.,  -54.],
        [-189.,  231.,  567.]],

       [[ 399., -392.,   84.],
        [ 450

As can be obsrved from the output above, there exist multiple rotations that correspond to a given $\Sigma$ number. This is usually the case when $\Sigma$ is large. In the following, we will compute the bycrystallography properties for one of the misorientations (e.g. corresponding to the second rotation in the array). The bycrystallography properties of interest are as follows:
+ The Coincidence Site Lattice (CSL). For more details, refer to this [tutorial](https://www.tf.uni-kiel.de/matwis/amat/def_en/kap_7/backbone/r7_1_2.html).
+ The DSC lattice that defines the **D**isplacements that are **S**ymmetry **C**onserving (also referred to as the **D**isplacement **S**hift **C**omplete). For more details, refer to this [tutorial](https://www.tf.uni-kiel.de/matwis/amat/def_en/kap_7/backbone/r7_1_3.html)
+ The boundary-plane symmetry group, and the symmetry axes. For more details refer to the following article:
    - [**Symmetries in the representation of grain boundary-plane distributions**](https://www.tandfonline.com/doi/full/10.1080/14786435.2012.722700?casa_token=4nDY3LLK9vQAAAAA%3AgECePf6JCdOPx6K_7cfxrJATdUAwMOSe3-F5NIdpjuI26uAPQfkkSTa61np1AWoDGQ3wcJrh3SS9). Patala, S., & Schuh, C. A. (2013). . Philosophical Magazine, 93(5), 524-573.

In [5]:
ct1 = 7
sig_id = str(sig_num)+'_'+str(ct1)
print(sig_id)

651_7


The $\Sigma$-misorientation is a rotation operation that is defined in the $P$ lattice reference frame and is computed as shown in the code snippet below. We use the variable `T_p1top2_p1` denote the **T**ransformation (`T`) from the $P1$ lattice to the $P2$ lattice (`p1top2`) expressed in the $P1$ reference frame (`p1`).

In [6]:
T_p1top2_p1 = s1['N'][ct1-1]/s1['D'][ct1-1]
print(T_p1top2_p1)

[[ 0.7281106  -0.46082949  0.29339478]
 [ 0.65437788  1.0921659   0.14132104]
 [-0.48387097  0.12903226  0.7311828 ]]


The **CSL** and the **DSC** lattice are determined using the `find_csl_dsc` function in the `byxtal.find_csl_dsc` module. The lattices are expressed using the basis vectors of the primitive cell $(P1)$. The following varibales are used:
+ `l_csl_p`: The lattice (`l`) of the CSL (`csl`) expressed in the primitive reference frame (`p`).
+ `l_dsc_p`: The lattice (`l`) of the DSC (`dsc`) expressed in the primitive reference frame (`p`).

In [7]:
# l_csl_p, l_dsc_p = fcd.find_csl_dsc(l_p_po, T_p1top2_p1, 1e-6)
# l_p_po = Matrix(l_p_po)
T_p1top2_p1 = np.array(T_p1top2_p1, dtype='double')
TI_p1top2_p1 = T_p1top2_p1*sig_num
TI_p1top2_p1 = ((np.around(TI_p1top2_p1)).astype(int))
l_csl_p = fcd.csl_finder_wSigma(TI_p1top2_p1, sig_num, l_p_po, 1e-6)
print(l_csl_p)
# # csl_mats[sig_id] = l_csl_p; 

[[-1. -1. 12.]
 [ 7. -2. -2.]
 [-3.  9. -1.]]


Now, we will determine the boundary-plane symmetry group (and the axes) for the $\Sigma$ misorientation. The following steps are followed:
+ First, we need to cnvert the rotation in the $P$ reference frame (`T_p1top2_p1`) to $PO$ reference frame (`T_p1top2_po1`).
+ The misorientation is converted to a quaternion using the `mat2quat` function in the `byxtal.tools` module.
+ The disorientation is obtained using the `misorient_fz` function in the `byxtal.misorient_fz` module.
+ The bicrystallography symmetry properties are computed using `disorient_symm_props` function in the `byxtal.disorient_symm_props` module. The outputs are stored in the following variables:
    - `bp_symm_grp`: The point group symmetry corresponding to the given misorientation.
    - `symm_grp_ax`: The axes of the symmetry point group are stored in a $3 \times 3$ matrix. The columns correspond to the $\hat{e}_x$, $\hat{e}_y$, $\hat{e}_z$ axes of the symmetry point group.

In [8]:
l_p_po = l1.l_p_po
l_po_p = nla.inv(l_p_po)
T_p1top2_po1 = np.dot(l_p_po, np.dot(T_p1top2_p1, l_po_p))

quat1 = bxt.mat2quat(T_p1top2_po1)
dis_quat1 = mfz.misorient_fz(quat1, l1.cryst_ptgrp)
x_g, y_g, z_g, bp_symm_grp = dsp.disorient_symm_props(dis_quat1, l1.cryst_ptgrp)
symm_grp_ax = (np.vstack((x_g, y_g, z_g))).transpose()
print(bp_symm_grp)
print(symm_grp_ax)

Cs
[[ 0.1108548   0.02534562  0.3048507 ]
 [-0.3048507   0.00921659  0.1108548 ]
 [ 0.         -0.10522273  0.0831411 ]]


One could compute the bycrystallography properties of a large number of $\Sigma$ rotation by using the code snippets discussed above and putting all of them together in a for-loop. This is shown in the code below.

In [10]:
n1 = 1; n2 = 40; sig_nums = 2*np.arange(n1,n2)+1;

num_sigs = 0;
sig_mats = {};
csl_mats = {};
# dsc_mats = {};
csl_bp_props = {};

for sig_num in sig_nums:
    s1 = cuf.csl_rotations(sig_num, sig_type, l1);
    for ct1 in range(np.shape(s1['N'])[0]):
        bp_symm_grp_props = {};
        sig_id = str(sig_num)+'_'+str(ct1+1);
        print(sig_id)
        
        #### Store the sigma-misorientation (in 'p' reference frame)
        T_p1top2_p1 = s1['N'][ct1]/s1['D'][ct1];
        sig_mats[sig_id] = T_p1top2_p1;

        # l_csl_p, l_dsc_p = fcd.find_csl_dsc(l_p_po, T_p1top2_p1, 1e-6)
        # l_p_po = Matrix(l_p_po)
        
        T_p1top2_p1 = np.array(T_p1top2_p1, dtype='double')
        TI_p1top2_p1 = T_p1top2_p1*sig_num
        TI_p1top2_p1 = (np.around(TI_p1top2_p1)).astype(int)
        l_csl_p = fcd.csl_finder_wSigma(TI_p1top2_p1, sig_num, l_p_po, 1e-6)
        csl_mats[sig_id] = l_csl_p;
        # dsc_mats[sig_id] = l_dsc_p;

        #### Generate boundary-planpe orientations
        l_p_po = l1.l_p_po; l_po_p = np.linalg.inv(l_p_po);
        T_p1top2_po1 = np.dot(l_p_po, np.dot(T_p1top2_p1, l_po_p));

        ## Find the corresponding disorientation
        quat1 = bxt.mat2quat(T_p1top2_po1); print(quat1);
        dis_quat1 = mfz.misorient_fz(quat1, l1.cryst_ptgrp); print(dis_quat1);
        x_g, y_g, z_g, bp_symm_grp = dsp.disorient_symm_props(dis_quat1, l1.cryst_ptgrp)
        symm_grp_ax = (np.vstack((x_g, y_g, z_g))).transpose();
        bp_symm_grp_props['symm_grp_ax'] = symm_grp_ax;
        bp_symm_grp_props['bp_symm_grp'] = bp_symm_grp;
        csl_bp_props[sig_id] = bp_symm_grp_props;

import pickle as pkl;
pkl_name = 'cF_Id_csl_common_rotations.pkl';
csl_props = {};
csl_props['sig_mats'] = sig_mats;
csl_props['csl_mats'] = csl_mats;
# csl_props['dsc_mats'] = dsc_mats;
csl_props['csl_bp_props'] = csl_bp_props;

jar = open(pkl_name, 'wb');
pkl.dump(csl_props, jar);
jar.close()

3_1
[0.8660254  0.28867513 0.28867513 0.28867513 1.        ]
[[0.8660254 ]
 [0.28867513]
 [0.28867513]
 [0.28867513]
 [1.        ]]
5_1
[ 9.48683298e-01  3.16227766e-01  2.92569456e-17 -3.65711820e-17
  1.00000000e+00]
[[ 9.48683298e-01]
 [ 3.16227766e-01]
 [ 2.92569456e-17]
 [-3.65711820e-17]
 [ 1.00000000e+00]]
7_1
[0.94491118 0.18898224 0.18898224 0.18898224 1.        ]
[[0.94491118]
 [0.18898224]
 [0.18898224]
 [0.18898224]
 [1.        ]]
9_1
[ 9.42809042e-01  2.35702260e-01  2.35702260e-01 -1.47196168e-17
  1.00000000e+00]
[[ 9.42809042e-01]
 [ 2.35702260e-01]
 [ 2.35702260e-01]
 [-1.47196168e-17]
 [ 1.00000000e+00]]
11_1
[ 9.04534034e-01  3.01511345e-01  3.01511345e-01 -3.06849434e-17
  1.00000000e+00]
[[ 9.04534034e-01]
 [ 3.01511345e-01]
 [ 3.01511345e-01]
 [-3.06849434e-17]
 [ 1.00000000e+00]]
13_1
[0.97072534 0.13867505 0.13867505 0.13867505 1.        ]
[[0.97072534]
 [0.13867505]
 [0.13867505]
 [0.13867505]
 [1.        ]]
13_2
[ 9.80580676e-01  1.96116135e-01 -5.66104887e-17